# GDC April 2021 Webinar: Using the GDC API

### Monday, April 26, 2021<br>2:00 PM - 3:00 PM (EST)<br>Bill Wysocki, Director of User Services <br>University of Chicago

## API User's Guide and Other GDC Helpful Links

[GDC API User's Guide](https://docs.gdc.cancer.gov/API/Users_Guide/Getting_Started/)

[GDC Support Website](https://gdc.cancer.gov/support)

GDC Helpdesk E-mail - support@nci-gdc.datacommons.io

Others?

## GDC API Overview

## GDC API Key Features

## Jupyter Notebook Overview

- links to JN resources? 
- Quick, couple sentence/1 paragraph summary
- Dependencies
- How to install dependencies from command line (or include a requirements.txt file?)
    - from windows, linux and macOS?

In [ ]:
#import packages
import requests
import json
import urllib